In [1]:
import numpy as np
import pandas as pd
import pint_pandas
from src.conditions import calc_gs_for_atmosphere, get_sound_speed
from src.conditions import calc_mach_from_cas, mach_from_temps, calc_ias_temp_diff
from src.asky import get_atmosphere_as_pd
from data.conc_data import get_cas_limit, get_mach_limit, get_total_temp_limit

## Get Weather

In [2]:
lat = '10-57-21.24S'
lon = '115-51-38.16E'
tude_unts = 'dms'

alts = list(range(40*10**3, 60*10**3 + 1, 1*10**3))
atmos = get_atmosphere_as_pd(lat, lon, alts, tude_units=tude_unts)

atmos.insert(loc=len(atmos.columns), column='SpeedSound', value=None)
for idx, row in atmos.iterrows():
    ss = get_sound_speed(
        altitude=row.Altitude,
        static_temp=row.Temperature)
    ss_units = ss.units
    atmos.iloc[idx, len(atmos.columns)-1] = ss.magnitude

atmos.SpeedSound = pint_pandas.PintArray(atmos.SpeedSound, dtype=ss_units)

atmos.insert(loc=len(atmos.columns), column='ISATempDiff', value=None)
for idx, row in atmos.iterrows():
    td = calc_ias_temp_diff(
        altitude=row.Altitude,
        static_temp=row.Temperature)
    td_units = td.units
    atmos.iloc[idx, len(atmos.columns)-1] = td.magnitude
atmos.ISATempDiff = pint_pandas.PintArray(atmos.ISATempDiff, dtype=td_units)

## Calcualte Airframe Speed Limits

In [3]:

atmos['CASLimit'] = pint_pandas.PintArray(
    get_cas_limit(atmos.Altitude), dtype='knots')
atmos['MachLimit'] = get_mach_limit(atmos.Altitude)
atmos['TotalTempLimit'] = get_total_temp_limit(atmos.Altitude)
atmos.TotalTempLimit = pint_pandas.PintArray(
    atmos.TotalTempLimit, dtype='degC')

atmos.insert(loc=len(atmos.columns), column='MachFromCASLimit', value=None)
for idx, row in atmos.iterrows():
    mach = calc_mach_from_cas(
        cas=row.CASLimit.to('m/s').magnitude,
        altitude=row.Altitude,
        static_temp=row.Temperature.to('degC').magnitude)
    atmos.iloc[idx, len(atmos.columns)-1] = mach.magnitude

atmos.MachFromCASLimit = atmos.MachFromCASLimit.astype('float64')

atmos['MachfromTotalTempLimit'] = mach_from_temps(atmos.TotalTempLimit,
                                                    atmos.Temperature)

## Calculate Max True Air Speed and Ground Speeds

In [4]:
atmos['MaxMach'] = atmos.loc[:, ['MachLimit', 'MachFromCASLimit', 'MachfromTotalTempLimit']].min(axis=1)
atmos['MaxTAS'] = atmos.SpeedSound * atmos.MaxMach
atmos['GS'] = calc_gs_for_atmosphere(atmos, atmos.MaxTAS, 100)
max_gs_row = atmos.GS.argmax()

In [ ]:
### Formatting Helpers

In [5]:
from IPython.display import display

# For setting column widths
# max_airport_name_len = df_airports['airport_name'].apply(
#     lambda x: len(str(x))).max()
# max_country_name_len = df_countries['country_name'].apply(
#     lambda x: len(str(x))).max()

# Formats
# text_col_names = r"name"
# text_cells_fmt = {'text-align': 'left'}
# mach_number_fmt = {'format': f'precision={3}'}
# country_name_w = {'width': f'{max_country_name_len/4}em'}

def highlight_greaterthan(s, column):
    print(s.index)
    max_row = s.argmax()
    print(max_row)
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] >= s.loc[column].max()
    print(is_max)
    return ['background-color: yellow' if is_max.any() else '' for v in is_max]

def display_df(ddf, df_caption):
    # Quick function to display formatted data frame
    display(
        ddf.style
        .format(thousands=",", precision=0)
        .format(precision=2,
                subset=ddf.columns[ddf.columns.str.contains('Mach')])
        .set_caption(df_caption)
        # .set_properties(
        #     subset=ddf.columns[ddf.columns.str.contains(text_col_names)],
        #     **text_cells_fmt)
        # .set_properties(
        #     subset=ddf.columns[ddf.columns.str.contains("Country")],
        #     **country_name_w)
        .set_table_styles(
            [dict(selector='th', props=[('text-align', 'right')]),
             dict(selector='caption',
                  props=[('text-align', 'left'),
                         ('font-size', '125%'),
                         ('line-height', '2.0')])])
        .hide(axis='index')
        .apply(lambda x: ['background: darkgreen' if x.name in [max_gs_row] 
                              else '' for i in x], 
                   axis=1)
    )



In [6]:
atmos_display = pd.DataFrame(atmos.Altitude.pint.to('ft').values.quantity.m, columns=['Altitude (ft)'])
atmos_display['Wind Dir'] = atmos.WindDirection.pint.to('deg').values.quantity.m
atmos_display['Wind Speed (kt)'] = atmos.WindSpeed.pint.to('knots').values.quantity.m
atmos_display['Static Temp (C)'] = atmos.Temperature.pint.to('degC').values.quantity.m
atmos_display['ISA +/- (C)'] = atmos.ISATempDiff.pint.to('delta_degC').values.quantity.m
# atmos_display['Pressure (hPa)'] = atmos.Pressure.pint.to('hPa').values.quantity.m
atmos_display['Local Speed Sound (m/s)'] = atmos.SpeedSound.pint.to('m/s').values.quantity.m
atmos_display['CAS Limit (kt)'] = atmos.CASLimit.pint.to('kt').values.quantity.m
# atmos_display['Mach Limit'] = atmos.MachLimit
# atmos_display['Total Temp Limit (C)'] = atmos.TotalTempLimit.pint.to('degC').values.quantity.m
atmos_display['CAS-Mach Limit'] = atmos.MachFromCASLimit
atmos_display['Total Temp-Mach Limit'] = atmos.MachfromTotalTempLimit
atmos_display['Max Mach'] = atmos.MaxMach
atmos_display['Max TAS (kt)'] = atmos.MaxTAS.pint.to('kt').values.quantity.m
atmos_display['Max GS (kt)'] = atmos.GS.pint.to('kt').values.quantity.m

In [7]:

display_df(atmos_display, "Current Atmosphere")

Altitude (ft),Wind Dir,Wind Speed (kt),Static Temp (C),ISA +/- (C),Local Speed Sound (m/s),CAS Limit (kt),CAS-Mach Limit,Total Temp-Mach Limit,Max Mach,Max TAS (kt),Max GS (kt)
"40,000",94,13,-54,2,297,494,1.50,2.03,1.50,865,878
"41,000",96,11,-56,0,295,506,1.57,2.05,1.57,900,911
"42,000",100,9,-62,-6,291,518,1.64,2.12,1.64,929,938
"43,000",106,7,-64,-8,290,530,1.72,2.14,1.72,968,975
"44,000",116,5,-66,-10,288,530,1.76,2.16,1.76,987,991
"45,000",160,3,-68,-12,287,530,1.80,2.19,1.80,"1,006","1,007"
"46,000",218,4,-70,-14,285,530,1.85,2.21,1.85,"1,026","1,024"
"47,000",238,7,-76,-20,281,530,1.91,2.27,1.91,"1,042","1,037"
"48,000",246,11,-78,-22,280,530,1.96,2.30,1.96,"1,065","1,056"
"49,000",250,14,-80,-24,278,530,2.01,2.32,2.01,"1,089","1,077"
